In [63]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
import locale
import sys
import os
import re

In [64]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [65]:
spark

# Leyendo lista de Colegios y Estudiantes

In [66]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [67]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [68]:
dfestu = {}
dfcole = {}
for i in coleDic:
    dfestu[i] = None
    dfcole[i] = None

In [69]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [70]:
print("Archivos de estudiantes: ", estuDic['20071'],estuDic['20072'])
print("Archivos de colegios: ",coleDic['2007'])

Archivos de estudiantes:  SB11_20071.txt SB11_20072.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2007.txt


In [71]:
columnasPuntos = [
 'cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']

In [72]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20071'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos)
df2 = spark.read.load(base2 + '/' + estuDic['20072'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos)

In [73]:
# SE unen los dos dataframes
columnasPuntosc = [
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']
df = unionAll(df1,df2)
for cp in columnasPuntosc:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    
df1 = df2 = None

In [74]:
print("Numero de filas ", df.count())

Numero de filas  502389


In [75]:
print(df.printSchema())

root
 |-- cole_cod_dane_institucion: string (nullable = true)
 |-- punt_lenguaje: float (nullable = true)
 |-- punt_matematicas: float (nullable = true)
 |-- punt_c_sociales: float (nullable = true)
 |-- punt_filosofia: float (nullable = true)
 |-- punt_biologia: float (nullable = true)
 |-- punt_quimica: float (nullable = true)
 |-- punt_fisica: float (nullable = true)
 |-- punt_ingles: float (nullable = true)
 |-- punt_profundizacion: float (nullable = true)
 |-- punt_interdisciplinar: float (nullable = true)

None


In [76]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('cole_cod_dane_institucion').count().orderBy('count',ascending=
False)

In [77]:
dsCountEst.show(10,False)

+-------------------------+-----+
|cole_cod_dane_institucion|count|
+-------------------------+-----+
|null                     |6981 |
|105001000108             |1088 |
|111001024732             |1013 |
|111001019411             |871  |
|105001013340             |856  |
|111102000753             |684  |
|305001003963             |669  |
|111001010839             |663  |
|168001003591             |660  |
|113001003274             |648  |
+-------------------------+-----+
only showing top 10 rows



In [78]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['punt_lenguaje']+df['punt_matematicas']+df['punt_c_sociales']+
                     df['punt_filosofia']+df['punt_biologia']+df['punt_quimica']+df['punt_fisica']+
                     df['punt_ingles']+df['punt_profundizacion']+df['punt_interdisciplinar'])

## Resultados finales por instituciòn

In [79]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = df.groupBy('cole_cod_dane_institucion').mean()
df = df.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df = df.orderBy('rank',ascending= True)

In [80]:
df.show(10,False)

+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+
|cole_cod_dane_institucion|avg(punt_lenguaje)|avg(punt_matematicas)|avg(punt_c_sociales)|avg(punt_filosofia)|avg(punt_biologia)|avg(punt_quimica) |avg(punt_fisica)  |avg(punt_ingles) |avg(punt_profundizacion)|avg(punt_interdisciplinar)|avg(Suma)        |rank|
+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+
|311001089221             |61.91473629600123 |70.93947340312756    |67.34420977140728   |51.50736838892887  |65.5378946003161  |58.80894711143092 |58.67578908016807 |76.65473837601512|6.891579000573409       |59.73578884

## Colegios

In [81]:
# se crea un dataframe con una columna del año.
from pyspark.sql.functions import lit
df = df.withColumn('año',lit(2007))
df.show(5,False)

+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|cole_cod_dane_institucion|avg(punt_lenguaje)|avg(punt_matematicas)|avg(punt_c_sociales)|avg(punt_filosofia)|avg(punt_biologia)|avg(punt_quimica) |avg(punt_fisica)  |avg(punt_ingles) |avg(punt_profundizacion)|avg(punt_interdisciplinar)|avg(Suma)        |rank|año |
+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|311001089221             |61.91473629600123 |70.93947340312756    |67.34420977140728   |51.50736838892887  |65.5378946003161  |58.80894711143092 |58.67578908016807 |76.65473837601512|6.891579000573409    

In [82]:
# SE crea Dataframe de colegios 2007
dfc = spark.read.load(base + '/' + coleDic['2007'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))
dfc = dfc.withColumn('COLE_CODMPIO_COLEGIO', dfc['COLE_CODMPIO_COLEGIO'].cast("string"))

In [83]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: string (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENT

In [84]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CODMPIO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [85]:
dfc = dfc.select(colsColegio)

In [86]:
dfc.show(5,False)

+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CODMPIO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|135442             |47001               |5                     |6           |5          |7            |4             |7               |8            |5          |null          |null         |
|135459             |25322               |6                     |7           |6          |6            |5             |7               |6            |5          |null          |null         |
|135467             |5001               

In [87]:
print("El numero de colegios: ", dfc.select('COLE_CODIGO_COLEGIO').count())

El numero de colegios:  9680


## Anàlisis comparativo entre estudiantes y colegios

In [88]:
dfjoin = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODIGO_COLEGIO)

In [89]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


In [90]:
dfjoin = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODMPIO_COLEGIO)
dfjoin.count()
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe cole_cod_dane_institucion en el archivo de colegios.